In [4]:
import json

with open('embeddings_output.json') as f:
    embed= json.load(f)

In [ ]:
embed[0]

In [5]:
len(embed[2]['embedding'])

384

# Generación de embeddings para sistema RAG
- **Objetivo**: Entregar embeddings generados a AI Devs para que lo implemente en sistema RAG del agente eCommerce
- **Proceso**: realizar desde el proceso de Chunking, evaluar los documentos generados para elegir el modelo, generar el embedding, evaluar resultados con implementación de Rag con OpenAI.

## Chunking
**Requisitos**
- *Tamaño óptimo:* 200-500 tokens por chunk
- *Overlap:* 50-100 tokens entre chunks consecutivos
- *Estrategias:*
  - Por párrafos (para políticas)
  - Por producto individual (para catálogos) (ya lo maneja equipo de AI devs)
  - Por pregunta-respuesta (para FAQs)

- Metadatos propuestos:
  - politica de reembolso, politica de envio
  -  Secciones: 

In [ ]:
from docx import Document
import json

doc= Document('docs/Recursos.docx')



In [23]:
# Cada párrafo
doc.paragraphs[3].text

'En kreadores.cl nos tomamos en serio la protección de tus derechos como consumidor. Por eso, esta política ha sido elaborada conforme a lo establecido en la Ley N° 19.496 sobre Protección de los Derechos de los Consumidores de Chile y en cumplimiento con los requisitos de Google Merchant Center.'

In [ ]:
# Extracción del texto
parrafos= []
for texto in doc.paragraphs:
    if texto.text: # si hay texto en la línea
        parrafos.append(texto.text.strip())

# Lo unimos en un solo string
texto_completo = "\n".join(parrafos)


Chunking - División en fragmentos

In [65]:
import re

# division por titulos numerados y separadores
chunks= re.split(r"(?:\n_{5,}\n|\n{2,}|(?=\n\d\.))",texto_completo)
chunks = [chunk.strip() for chunk in chunks if chunk.strip()]



In [ ]:
len(chunks)

15

In [101]:
tamanio= []
for i in chunks:
    tamanio.append(len(i))
print(f'Máxima longitud de chunk generado: {max(tamanio)}')

Máxima longitud de chunk generado: 697


In [66]:
chunks

['Política de reembolso:\nPolítica de reembolso\nPolítica de Reembolsos, Cambios y Devoluciones\nNew Line Chile SpA | Kreadores.cl\nÚltima actualización: 11-06-2025\nEn kreadores.cl nos tomamos en serio la protección de tus derechos como consumidor. Por eso, esta política ha sido elaborada conforme a lo establecido en la Ley N° 19.496 sobre Protección de los Derechos de los Consumidores de Chile y en cumplimiento con los requisitos de Google Merchant Center.\nEsta política aplica a todas las compras realizadas a través de nuestro sitio web: https://www.kreadores.cl',
 '1. Derecho a Retracto\nSegún lo dispuesto en el artículo 3 bis de la Ley del Consumidor chilena, si compraste un producto por internet y fue entregado a domicilio, tienes derecho a retractarte de la compra dentro de un plazo de 30 días corridos desde la compra, siempre que:\nEl producto no haya sido usado, abierto ni dañado.\nEsté en su empaque original, sellado y en perfectas condiciones.\nSe acompañe con la boleta o co

Se añade la metadata

In [ ]:
import re


politicas= { 'Politica de reembolso':['1. Derecho a Retracto',
 '2. Garantía Legal por Producto Defectuoso',
 '3. Cambios Voluntarios',
 '4. Productos Excluidos de Reembolso o Cambio',
 '5. Procedimiento de Devolución',
 '6. Reembolsos','7. Costos de Envío en Devoluciones',
 '8. Dudas y Contacto'], 
 'Politica de envio': [ '1. Despachos a Nivel Nacional',
 '2. Plazos de despacho y entrega',
 '3. Horario de corte para procesamiento de pedidos',
 '4. Seguimiento de envío',
 '5. Responsabilidad del cliente',
 '6. Problemas con la entrega']}

chunks_metadata= []
for chunk in chunks:
    match_seccion= re.search(r"^\d+\.\s+.+", chunk)
    if match_seccion:
        seccion= match_seccion.group(0)
    else:
        seccion='General'

    for politica, secciones in politicas.items():
        if seccion in secciones:
            politica_metadata= politica
            break
        else:
            politica_metadata='Otros'

    # agrega la metadata
    chunks_metadata.append( {
        'text': chunk,
        'metadata': {
            'politica':politica_metadata,
            'seccion': seccion
        }
    })


Añadiendo los FAQ

In [ ]:
# como son pocas en una lista
faqs = [
    "¿Cuándo llegará el pedido a mi casa?",
    "¿Qué productos debo comprar? acá es indispensable preguntar para qué se usará el equipo y qué presupuesto se dispone.",
    "¿Quiero saber si hacen colaboración o canje?",
    "¿Este producto es compatible con mi cámara? Acá debemos saber qué cámara tiene el cliente para saber si es compatible o no.",
    "¿Me podrían hacer un descuento?"
]

# se agregan como chunks adicionales
for pregunta in faqs:
    chunks_metadata.append({
        "text": pregunta,
        "metadata": {
            "politica": "FAQ",
            "seccion": "Pregunta Frecuente"
        }
    })


In [155]:
chunks_metadata[0]

{'text': 'Política de reembolso:\nPolítica de reembolso\nPolítica de Reembolsos, Cambios y Devoluciones\nNew Line Chile SpA | Kreadores.cl\nÚltima actualización: 11-06-2025\nEn kreadores.cl nos tomamos en serio la protección de tus derechos como consumidor. Por eso, esta política ha sido elaborada conforme a lo establecido en la Ley N° 19.496 sobre Protección de los Derechos de los Consumidores de Chile y en cumplimiento con los requisitos de Google Merchant Center.\nEsta política aplica a todas las compras realizadas a través de nuestro sitio web: https://www.kreadores.cl',
 'metadata': {'politica': 'Otros', 'seccion': 'General'}}

## Implementación del modelo y embedding

Características principales del documento:
- Texto
- Español

In [ ]:
from fastembed import TextEmbedding

# modelos disponibles
modelos= TextEmbedding.list_supported_models()
modelos

In [113]:
for modelo in modelos:
    if 'English'not in modelo['description']:
        print(modelo['model'],modelo['description'], '\n',modelo['dim'], '\n')

BAAI/bge-small-zh-v1.5 Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year. 
 512 

sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 Text embeddings, Unimodal (text), Multilingual (~50 languages), 512 input tokens truncation, Prefixes for queries/documents: not necessary, 2019 year. 
 384 

sentence-transformers/paraphrase-multilingual-mpnet-base-v2 Text embeddings, Unimodal (text), Multilingual (~50 languages), 384 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year. 
 768 

intfloat/multilingual-e5-large Text embeddings, Unimodal (text), Multilingual (~100 languages), 512 input tokens truncation, Prefixes for queries/documents: necessary, 2024 year. 
 1024 

jinaai/jina-embeddings-v3 Multi-task unimodal (text) embedding model, multi-lingual (~100), 1024 tokens truncation, and 8192 sequence length. Prefixes for queries/documents: not necessary, 2024 year. 
 1024 



Como solicitan una dimensionalidad de 1536, y como los vectores y la query deben tener la mismas dimensiones. Entonces se descarta uso de FastEmbed. 
Se procede a evaluar OpenAi.

Modelos OPENAI
- 'text-embedding-3-small': 1536 (*modelo seleccionado*)
- 'text-embedding-3-large': 3072

Estructura del embedding:
- Cada embedding se guarda con:
  - Vector numérico (1536 dimensiones típicamente)
  - Texto original
  - Metadatos (categoría, fuente, fecha)


In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
api_key=os.getenv('OPENAI_API_KEY')
cliente= OpenAI()
cliente

In [156]:
embedding_recursos=[]
modelo_seleccionado = "text-embedding-3-small"
for chunk in chunks_metadata:
    response = cliente.embeddings.create(
        input= chunk['text'], 
        model = modelo_seleccionado
    )

    embedding= response.data[0].embedding

    embedding_recursos.append({
        'vector': embedding, 
        'text': chunk['text'], 
        'metadata':chunk['metadata']
    })



In [157]:
print('Dimensiones del vector:', len(embedding_recursos[0]['vector']))

Dimensiones del vector: 1536


Exportar los embedding

In [166]:
with open("json_embedding/embeddings.json", "w", encoding="utf-8") as file:
    json.dump(embedding_recursos, file, ensure_ascii=False, indent=2)

## Carga en Qdrant

In [168]:
import json
with open('json_embedding/embeddings.json') as f:
    embed_recursos= json.load(f)

In [144]:
# creacion del cliente
from qdrant_client import QdrantClient, models

# primero correr el contenedor de Docker qdrant
qdrant_cliente= QdrantClient(' http://localhost:6333')
qdrant_cliente

In [163]:
# Creación de los puntos
# modelo_seleccionado = "text-embedding-3-small"
points= []
id = 0

for elemento_embedding in embed_recursos:
    
    point= models.PointStruct(
        id= id, 
        vector= elemento_embedding['vector'],
        payload = {
            "text": elemento_embedding['text'], 
            'policies': elemento_embedding['metadata']['politica'],
            'section': elemento_embedding['metadata']['seccion']
        }
        
    )
    points.append(point)
    id +=1


In [169]:
# creacion de la coleccion en qdrant
nombre_coleccion='prueba_embedding_recursos'
dimension= 1536

qdrant_cliente.create_collection(
    collection_name= nombre_coleccion, 
    vectors_config= models.VectorParams(
        size= dimension, 
        distance= models.Distance.COSINE
    )
)

True

In [170]:
# indexacion del payload
qdrant_cliente.create_payload_index(
    collection_name= nombre_coleccion,
    field_name= 'policies', 
    field_schema= "keyword"
)

qdrant_cliente.create_payload_index(
    collection_name= nombre_coleccion,
    field_name= 'section', 
    field_schema= "keyword"
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [172]:
# se insertan en la coleccion
qdrant_cliente.upsert(collection_name= nombre_coleccion, points= points)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

## Prueba de validación en sistema RAG

Se crea un sistema RAG básico par validar información embebida

In [186]:
def vector_search(query):
    # embedding de la query
    query_response= cliente.embeddings.create(
        input= query,
        model= modelo_seleccionado
    )
    query_embed = query_response.data[0].embedding

    # busqueda entre query y vectores almacenados
    resultados= qdrant_cliente.query_points(
        collection_name= nombre_coleccion,
        query= query_embed, 
        limit= 5, 
        with_payload=True
    )
    lista_resultados=[]
    for i in resultados.points:
        lista_resultados.append(i.payload)

    return lista_resultados

In [206]:
rpta= vector_search('Tengo derecho a retracto?')
rpta

[{'text': '1. Derecho a Retracto\nSegún lo dispuesto en el artículo 3 bis de la Ley del Consumidor chilena, si compraste un producto por internet y fue entregado a domicilio, tienes derecho a retractarte de la compra dentro de un plazo de 30 días corridos desde la compra, siempre que:\nEl producto no haya sido usado, abierto ni dañado.\nEsté en su empaque original, sellado y en perfectas condiciones.\nSe acompañe con la boleta o comprobante de compra.\nNo haya sido adquirido en liquidación o con descuentos especiales.\nEn este caso, los costos de cambio son gratuitos.',
  'policies': 'Politica de reembolso',
  'section': '1. Derecho a Retracto'},
 {'text': '3. Cambios Voluntarios\nAdemás del derecho a retracto y la garantía legal, aceptamos cambios voluntarios dentro de un plazo de 10 días desde la entrega, siempre que:\nEl producto no haya sido usado ni dañado.\nSe encuentre en perfectas condiciones, con todos sus accesorios y empaques.\nNo haya sido adquirido en liquidación o con des

In [ ]:
def build_prompt(query, result_from_search):

    prompt_template= """
    Eres asistente de MiTienda, tienda online eCommerce. Responde la PREGUNTA basado en el CONTEXTO. Usa SOLO la información de la base de datos. No investes información, si no cuentas con la respuesta mencionalo.
    Habla en primera persona.
PREGUNTA: {question}
    
CONTEXTO:
{context}
    """.strip()
    contexto= ""
    for i in result_from_search:
        contexto= contexto + f"Text: {i['text']} \nPolítica: {i['policies']} \nSección: {i['section']}\n\n "
    
    prompt= prompt_template.format(question= query, context= contexto)
    return prompt

In [207]:
prompt= build_prompt('Tengo derecho a retracto?',rpta )
print(prompt)

Eres asistente de MiTienda, tienda online eCommerce. Responde la PREGUNTA basado en el CONTEXTO. Usa solo la información de la base de datos.

PREGUNTA: Tengo derecho a retracto?

CONTEXTO:
Text: 1. Derecho a Retracto
Según lo dispuesto en el artículo 3 bis de la Ley del Consumidor chilena, si compraste un producto por internet y fue entregado a domicilio, tienes derecho a retractarte de la compra dentro de un plazo de 30 días corridos desde la compra, siempre que:
El producto no haya sido usado, abierto ni dañado.
Esté en su empaque original, sellado y en perfectas condiciones.
Se acompañe con la boleta o comprobante de compra.
No haya sido adquirido en liquidación o con descuentos especiales.
En este caso, los costos de cambio son gratuitos. 
Política:Politica de reembolso 
Sección: 1. Derecho a Retracto

 Text: 3. Cambios Voluntarios
Además del derecho a retracto y la garantía legal, aceptamos cambios voluntarios dentro de un plazo de 10 días desde la entrega, siempre que:
El produc

In [ ]:
#prueba de api
def output_llm(prompt):

    response= cliente.chat.completions.create(
        model= 'gpt-4o-mini', 
        messages= [
            {
                "role":"user", 
                "content":  prompt
            }
        ]
    )
    return response.choices[0].message.content

In [203]:
rpta= output_llm('how can i get the certificate?')
rpta

"To provide you with the best advice on how to obtain a certificate, I'll need a bit more information about the specific type of certificate you're interested in. Here are a few common types:\n\n1. **Academic Certificates**: If you're looking to obtain a certificate related to education, such as from a college or university, you typically need to enroll in a course or program, complete the required coursework, and pass any necessary assessments.\n\n2. **Professional Certifications**: For professional certifications (like PMP, ITIL, etc.), you usually need to meet certain prerequisites (like work experience), complete a training program, and pass an exam.\n\n3. **Online Course Certificates**: Many online platforms (like Coursera, edX, etc.) offer certificates upon completion of their courses. You generally need to sign up, complete the course requirements, and sometimes pay a fee.\n\n4. **Birth or Marriage Certificates**: If you need official documents like these, you typically request 

In [210]:
def rag_vector_search(query):
    resultados_busqueda=vector_search(query)
    prompt= build_prompt(query,resultados_busqueda)
    respuesta= output_llm(prompt)

    return respuesta


 Ejemplo de pregunta:  
- INFORMACIÓN RELEVANTE:
    1. Tienes 30 días para devolver productos desde la fecha de entrega
    2. Los productos deben estar en perfecto estado, sin uso
    3. El cliente paga el costo del envío de devolución

- PREGUNTA: ¿Puedo devolver un producto?

In [219]:
query= '¿Puedo devolver un producto?'
respuesta_llm= rag_vector_search(query)

In [220]:
respuesta_llm

'Sí, puedes devolver un producto. Debes solicitar la devolución escribiendo a ayuda@kreadores.cl, indicando tu número de pedido, el motivo y pruebas como fotos si es necesario. Evaluaremos tu solicitud en un máximo de 5 días hábiles. Si se aprueba, te proporcionaré la dirección para devolver el producto. Ten en cuenta que el cliente asume el costo del envío de devolución, salvo en casos de productos defectuosos o errores de despacho. Además, para realizar cambios voluntarios, el producto debe estar en perfectas condiciones y no usado, y el plazo para esto es de 10 días desde la entrega.'

In [214]:
print(respuesta_llm)

Sí, puedes devolver un producto, siempre y cuando cumplas con ciertos requisitos. Tienes derecho a retracto dentro de 30 días desde la compra si el producto no ha sido usado ni dañado, está en su empaque original y sellado, y se acompaña con la boleta o comprobante de compra. En este caso, los costos de envío de devolución son gratuitos.

Si deseas realizar un cambio voluntario, puedes hacerlo dentro de un plazo de 10 días desde la entrega, siempre que el producto esté en perfectas condiciones y sin daños. Sin embargo, en este caso, los costos de envío corren por cuenta del cliente.

Para iniciar el proceso de devolución, debes escribir a ayuda@kreadores.cl con tu número de pedido, el motivo de la devolución y cualquier prueba necesaria (como fotos). Te informarán la dirección para el retorno una vez que tu solicitud sea aprobada.


In [218]:
query= '¿Hacen envíos a Argentina?'
respuesta_llm= rag_vector_search(query)
print(respuesta_llm)

Lamentablemente, no realizamos envíos a Argentina. Solo hacemos despachos a nivel nacional en Chile. Si necesitas más información sobre nuestros servicios, no dudes en preguntarme.
